## Análsis de datos - Clase 1

### Carga de datos y visualización rápida

In [92]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from sqlalchemy import create_engine, MetaData
import json
from io import StringIO

1. Cargar datos desde un archivo CSV a un df de Pandas

In [107]:

df_csv = pd.read_csv("https://people.sc.fsu.edu/~jburkardt/data/csv/airtravel.csv")

# Otros formatos comunes: pd.read_parquet, pd.read_json, pd.read_html

In [ ]:
# Mostrar las primeras filas (las 5 primeras por defecto)
df_csv.head()  

,Month,"""1958""","""1959""","""1960"""
0,JAN,340,360,417
1,FEB,318,342,391
2,MAR,362,406,419
3,APR,348,396,461
4,MAY,363,420,472


In [109]:
df_csv.tail(6) # Mostrar las últimas 6 filas

,Month,"""1958""","""1959""","""1960"""
6,JUL,491,548,622
7,AUG,505,559,606
8,SEP,404,463,508
9,OCT,359,407,461
10,NOV,310,362,390
11,DEC,337,405,432


2. Recolectar datos a través de una API

In [ ]:

# Ejemplo: API del SMN Argentino

url_smn = "https://ws.smn.gob.ar/map_items/forecast/1" # Forecast de 1 día
response_smn = requests.get(url_smn)

if response_smn.status_code == 200:
    data_smn = response_smn.json()
    print("Se obtuvo respuesta!")
    
else:
    print(f"Se produjo un error: {response_smn.status_code}")

Se obtuvo respuesta!


In [147]:
data_smn[:2]

[{'_id': '609d947e818e15902c74d831',
  'dist': 20.55,
  'lid': 9539,
  'fid': 9539,
  'name': 'Tunuyan',
  'province': 'Mendoza',
  'lat': '-33.58210373',
  'lon': '-69.02268982',
  'zoom': '2',
  'updated': 1557349200,
  'weather': {'day': 1,
   'morning_temp': 10,
   'morning_id': 2,
   'morning_desc': 'Cielo parcialmente nublado. Vientos leves del sector sur.',
   'afternoon_temp': 22,
   'afternoon_id': 14,
   'afternoon_desc': 'Nubosidad variable. Probabilidad de tormentas aisladas. Vientos leves del sector sur.'}},
 {'_id': '609d947e818e15902c74d836',
  'dist': 7.12,
  'lid': 9553,
  'fid': 9553,
  'name': 'San Rafael',
  'province': 'Mendoza',
  'lat': '-34.60757828',
  'lon': '-68.32798004',
  'zoom': '2',
  'updated': 1557349200,
  'weather': {'day': 1,
   'morning_temp': 13,
   'morning_id': 4,
   'morning_desc': 'Cielo nublado. Probabilidad de lluvias y tormentas, algunas fuertes. Vientos leves del sector norte, cambiando a moderados o regulares del sector sur con ráfagas.',

In [ ]:
# Primera forma 

df_api_1 = pd.DataFrame(data_smn)

df_api_1.head()

,_id,dist,lid,fid,name,province,lat,lon,zoom,updated,weather
0,609d947e818e15902c74d831,20.55,9539,9539,Tunuyan,Mendoza,-33.58210373,-69.02268982,2,1557349200,"{'day': 1, 'morning_temp': 10, 'morning_id': 2..."
1,609d947e818e15902c74d836,7.12,9553,9553,San Rafael,Mendoza,-34.60757828,-68.32798004,2,1557349200,"{'day': 1, 'morning_temp': 13, 'morning_id': 4..."
2,609d947e818e15902c74d83b,77.99,9560,9560,General Alvear,Mendoza,-34.9721489,-67.68930054,2,1557349200,"{'day': 1, 'morning_temp': 11, 'morning_id': 4..."
3,609d947e818e15902c74d840,66.80,4164,4164,San Bernardo,Buenos Aires,-36.69836044,-56.67993927,2,1557306000,"{'day': 1, 'morning_temp': 12, 'morning_id': 1..."
4,609d947e818e15902c74d844,64.47,9629,9629,Apóstoles,Misiones,-27.91513062,-55.75431442,2,1557349200,"{'day': 1, 'morning_temp': 19, 'morning_id': 4..."


In [ ]:
# Segunda forma 

df_api_2 = pd.json_normalize(data_smn) # Desarma los diccionarios anidados

df_api_2.head()

,_id,dist,lid,fid,name,province,lat,lon,zoom,updated,weather.day,weather.morning_temp,weather.morning_id,weather.morning_desc,weather.afternoon_temp,weather.afternoon_id,weather.afternoon_desc
0,609d947e818e15902c74d831,20.55,9539,9539,Tunuyan,Mendoza,-33.58210373,-69.02268982,2,1557349200,1,10,2,Cielo parcialmente nublado. Vientos leves del ...,22,14,Nubosidad variable. Probabilidad de tormentas ...
1,609d947e818e15902c74d836,7.12,9553,9553,San Rafael,Mendoza,-34.60757828,-68.32798004,2,1557349200,1,13,4,Cielo nublado. Probabilidad de lluvias y torme...,18,3,Nubosidad variable. Probabilidad de lluvias y ...
2,609d947e818e15902c74d83b,77.99,9560,9560,General Alvear,Mendoza,-34.9721489,-67.68930054,2,1557349200,1,11,4,Cielo nublado. Probabilidad de lluvias y torme...,20,3,Nubosidad variable. Probabilidad de lluvias y ...
3,609d947e818e15902c74d840,66.80,4164,4164,San Bernardo,Buenos Aires,-36.69836044,-56.67993927,2,1557306000,1,12,10,Nublado,19,11,Inestable con probables precipitaciones
4,609d947e818e15902c74d844,64.47,9629,9629,Apóstoles,Misiones,-27.91513062,-55.75431442,2,1557349200,1,19,4,Probables neblinas y bancos de niebla. Nubosid...,26,4,Nubosidad variable. Probabilidad de lluvias y ...


In [90]:
# Tercera forma

json_str = json.dumps(data_smn) # Convierte data_smn que es una lista
json_str_io = StringIO(json_str) # StringIO es el tipo de entrada que espera pd.read_json
df_api_3 = pd.read_json(json_str_io)

df_api_3.head()


,_id,dist,lid,fid,name,province,lat,lon,zoom,updated,weather
0,609d947e818e15902c74d831,20.55,9539,9539,Tunuyan,Mendoza,-33.582104,-69.022690,2,1557349200,"{'day': 1, 'morning_temp': 10, 'morning_id': 2..."
1,609d947e818e15902c74d836,7.12,9553,9553,San Rafael,Mendoza,-34.607578,-68.327980,2,1557349200,"{'day': 1, 'morning_temp': 13, 'morning_id': 4..."
2,609d947e818e15902c74d83b,77.99,9560,9560,General Alvear,Mendoza,-34.972149,-67.689301,2,1557349200,"{'day': 1, 'morning_temp': 11, 'morning_id': 4..."
3,609d947e818e15902c74d840,66.80,4164,4164,San Bernardo,Buenos Aires,-36.698360,-56.679939,2,1557306000,"{'day': 1, 'morning_temp': 12, 'morning_id': 1..."
4,609d947e818e15902c74d844,64.47,9629,9629,Apóstoles,Misiones,-27.915131,-55.754314,2,1557349200,"{'day': 1, 'morning_temp': 19, 'morning_id': 4..."


3. Conectarse a una base de datos

In [110]:
DATABASE_URL = "sqlite:///../datasets/movie.sqlite"  # Path al archivo sqlite
engine = create_engine(DATABASE_URL)

# Obtener datos de las tablas
metadata = MetaData()
metadata.reflect(bind=engine)

print("Tablas disponibles en la DB:")
print(list(metadata.tables.keys()))

Tablas disponibles en la DB:
['IMDB', 'earning', 'genre']


In [111]:
# Ejemplo: query a la tabla IMDB
query_imdb = "SELECT * FROM IMDB"

# Convertir a df de Pandas

df_sql_imdb = pd.read_sql(query_imdb, engine)
df_sql_imdb.head()


,Movie_id,Title,Rating,TotalVotes,MetaCritic,Budget,Runtime,CVotes10,CVotes09,CVotes08,...,Votes3044,Votes3044M,Votes3044F,Votes45A,Votes45AM,Votes45AF,VotesIMDB,Votes1000,VotesUS,VotesnUS
0,36809,12 Years a Slave (2013),8.1,496092,96,20000000.0,134 min,75556,126223,161460,...,8.0,7.9,8.0,7.8,7.8,8.1,8.0,7.7,8.3,8.0
1,30114,127 Hours (2010),7.6,297075,82,18000000.0,94 min,28939,44110,98845,...,7.5,7.5,7.5,7.3,7.3,7.5,7.6,7.0,7.7,7.6
2,37367,50/50 (2011),7.7,283935,72,8000000.0,100 min,28304,47501,99524,...,7.6,7.6,7.6,7.4,7.4,7.5,7.4,7.0,7.9,7.6
3,49473,About Time (2013),7.8,225412,,12000000.0,123 min,38556,43170,70850,...,7.6,7.6,7.7,7.6,7.5,7.8,7.7,6.9,7.8,7.7
4,14867,Amour (2012),7.9,76121,94,8900000.0,127 min,11093,15944,22942,...,7.7,7.7,7.9,7.9,7.8,8.1,6.6,7.2,7.9,7.8


4. Web scraping

In [146]:
url = "https://news.ycombinator.com/"
#url = "https://en.wikipedia.org/wiki/Web_scraping"
response = requests.get(url) # Buscar el contenido HTML 

if response.status_code == 200:
    #print(response.text[:800])  # First 500 characters of the response

    soup = BeautifulSoup(response.text, "html.parser") # Parsear el contenido
    #print(soup.prettify()[:1000])  # First 1000 characters of the parsed HTML

    #title = soup.select_one('h1').text
    #text = soup.select_one('p').text
    link = soup.select_one('a').get('href')

    #print(title)
    #print(text)
    print(link)


    #titles = [title.text for title in soup.find_all("a", class_="titlelink")]

else:
    print(f"HTML no disponible (Error:{response.status_code})")

https://news.ycombinator.com


In [140]:
print("Top 5 Headlines:")
print(titles[:5])

Top 5 Headlines:
[]


### Introducción al análsis más detallado

1. Ejemplo con dataset "turismo.csv"

In [106]:
path="../datasets/turismo.csv"

df = pd.read_csv(path)

df.head()

,indice_tiempo,turismo_receptivo,turismo_emisivo,saldo
0,2010-01-01,254.157,162.680,91.477
1,2010-02-01,186.441,186.227,0.214
2,2010-03-01,203.821,167.262,36.559
3,2010-04-01,207.516,141.750,65.766
4,2010-05-01,191.665,154.430,37.235


2. Revisar datos faltantes

In [122]:
print(f"Valores faltantes: {df.isnull().sum()}")  # Calcular valores faltantes


Valores faltantes: indice_tiempo        0
turismo_receptivo    0
turismo_emisivo      0
saldo                0
dtype: int64


In [121]:
print(f"Columnas del dataframe:\n{df.dtypes}\n")
print("Cantidad de datos duplicados:", df.duplicated().sum())

Columnas del dataframe:
indice_tiempo         object
turismo_receptivo    float64
turismo_emisivo      float64
saldo                float64
dtype: object

Cantidad de datos duplicados: 0
